Исламова Сания Маратовна
НПИмд-01-24
1132249576

Семинар
1 пункт

In [11]:
import tensorflow as tf

def transformation_network(input_shape):
    inputs = tf.keras.layers.Input(shape=input_shape)
    
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Flatten()(x)
    
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    
    outputs = tf.keras.layers.Dense(2)(x)  
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    return model
input_shape = (28, 28, 1)  
model = transformation_network(input_shape)
model.summary()


Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1600)              0   

2 пункт

In [3]:
import cv2


image = cv2.imread('image.jpg')


object1_x, object1_y = 100, 150
object2_x, object2_y = 200, 250


if object1_x < object2_x:
   print('Object 1 is to the left of Object 2')
elif object1_x > object2_x:
   print('Object 1 is to the right of Object 2')
else:
   print('Object 1 and Object 2 are in the same column')

if object1_y < object2_y:
    print('Object 1 is above Object 2')
elif (object1_y > object2_y):
    print('Object 1 is below Object 2')
else:
  print('Object 1 and Object 2 are in the same row')

Object 1 is to the left of Object 2
Object 1 is above Object 2


3 пункт

In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

dataset_path = "/Users/4eka0/Downloads/CLEVR_v1.0"
clevr_dataset = image_dataset_from_directory(dataset_path, image_size=(128, 128), batch_size=32)
import json

# Путь к аннотациям
path_to_annotations = "/Users/4eka0/Downloads/CLEVR_v1.0/scenes/CLEVR_train_scenes.json"

# Загрузка данных
with open(path_to_annotations, 'r') as f:
    clevr_data = json.load(f)

# Структура данных
scenes = clevr_data['scenes']  
print(scenes[0])

Found 100000 files belonging to 3 classes.
{'image_index': 0, 'objects': [{'color': 'blue', 'size': 'large', 'rotation': 269.8517172617167, 'shape': 'cube', '3d_coords': [-1.3705521821975708, 2.0794010162353516, 0.699999988079071], 'material': 'rubber', 'pixel_coords': [269, 88, 12.661545753479004]}, {'color': 'green', 'size': 'large', 'rotation': 292.2219458666971, 'shape': 'cylinder', '3d_coords': [-2.9289753437042236, -1.7488206624984741, 0.699999988079071], 'material': 'metal', 'pixel_coords': [93, 108, 11.522202491760254]}, {'color': 'cyan', 'size': 'small', 'rotation': 25.545135239473026, 'shape': 'cube', '3d_coords': [1.5515961647033691, 0.6776641607284546, 0.3499999940395355], 'material': 'rubber', 'pixel_coords': [319, 162, 10.045343399047852]}, {'color': 'brown', 'size': 'large', 'rotation': 327.3489188814305, 'shape': 'cylinder', '3d_coords': [-0.25301405787467957, -2.3089325428009033, 0.699999988079071], 'material': 'metal', 'pixel_coords': [132, 159, 9.392304420471191]}, {